In [1]:
!pip install git+https://github.com/openai/whisper.git gradio
!pip install transformers

import whisper
from transformers import BartForConditionalGeneration, BartTokenizer
import gradio as gr


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-0mgiyfwy
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-0mgiyfwy
  Resolved https://github.com/openai/whisper.git to commit 423492dda7806206abe56bdfe427c1096473a020
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# **Code for ASR and Summarizer in a pipline**


In [3]:

asr_model = whisper.load_model("base")
summarization_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

def transcribe_audio(audio_file):
    result = asr_model.transcribe(audio_file)
    return result['text']

def summarize_text(text, max_length=100, min_length=35):
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = summarization_model.generate(inputs["input_ids"], max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def process_audio(audio_file):
    transcription = transcribe_audio(audio_file)
    summary = summarize_text(transcription)
    return transcription, summary



/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
/usr/local/li

# **Gradio to display the summary and take the input:**

In [4]:
interface = gr.Interface(
    fn=process_audio,
    inputs=gr.Audio(type="filepath"),
    outputs=[
        gr.Textbox(label="Transcription"),
        gr.Textbox(label="Summary")
    ],
    title="Doctor-Patient Conversation Summarizer",
    description="Upload an audio file of a doctor-patient conversation to get a transcription and summary."
)

interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://17e4a8ddad0b65d0f3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
